# Skull AI

The objective here is to make AI/Bots that can play the deception board/tile game: Skull. See this link [LINK] to a video showing how to play and then the flow diagram in the gitrepo showing the decision flow for  a player.

ASSUMPTIONS:
 - Only 4 players in a game
 - for now, wont pay attention to previous rounds
 - simple terms (linear)

### Library imports

In [1]:
import random
from numpy.random import rand, choice
import numpy as np
import pandas as pd

### Player and Game classes

Player class.
Decisions for the bot to make:
 - What tile to place first
 - To bet or place another tile
     - binary classifications
 - What tile to place [DONE]
     - binary classification
 - what to bet
     - multiclass classification/ integer prediction
 - to bet or pass
     - binary classification
 - who to pick when seeking
     - multiclass classification
 
Defining a class for the player:
 - a player needs to be able to place a flower and skull [DONE]
 - a player needs to be able to remove a tile
     - randomly if they're skulled by somone else [DONE]
     - choose if they skull themselves
 - 

In [95]:
class skullPlayer:
    def __init__(self):
        self.flowers=3
        self.skull=1
        self.totalTiles=4
        self.hasSkull=True
        self.placedTiles=[]
        self.point=0
        self.theta_tp=(rand(1,30)*2)-1
        self.theta_bp=(rand(1,31)*2)-1
        self.theta_w2b=(rand(12,95)*2-1)
        
    def summary(self):
        print("Flowers: {}\nSkull: {}\nPoint: {}\nPlaced tiles: {}".format(self.flowers,self.skull,self.point, self.placedTiles))
    
    def placeFlower(self):
        if len(self.placedTiles)<self.totalTiles:
            if (len(self.placedTiles)-sum(self.placedTiles))<self.flowers:
                self.placedTiles.append(False)
                
    def placeSkull(self):
        if len(self.placedTiles)<self.totalTiles:
            if self.hasSkull & (sum(self.placedTiles)==0):
                self.placedTiles.append(True)
                
    def reset(self):
        self.placedTiles=[]
                
    def removeRandom(self):
        if self.hasSkull:
            if choice([True]+[False]*self.flowers):
                self.skull+=-1
                self.hasSkull=False
            else:
                self.flowers+=-1
        else:
            self.flowers+=-1
        self.totalTiles+=-1
        self.reset()
    
    def chooseTileToPlace(self,externalInfo):
        """
        Function to decide what tile to place.
        External info should have for each player:
        - If they have a point (binary)
        - How many tiles they have (numerical)
        - How many tiles they have placed (numerical)
        - If they have their skull (dual-binary) [1,0] skull, [0,0] unknown, [0,1] no skull 
        - their place in the turn order (numerical)
        - if they're out of the game (binary) [1] out, [0] in
        - optional (not implemented)
            - if they just lost a tile
            - if they just won a point
        LAST row should be that on the "bot"
        Extra
        - random float for "randomness"
        - bias term
        therefore external info is a 1x30 vector
        
        RETURNS
        boolean. true, place a skull, false place a flower
        """
        
        inVec=np.reshape(y.externalInfo,(-1,1))
        inVec=np.concatenate((inVec,np.array([[rand()],[1]])))
        
        skull = ((1/(1+np.exp(-np.dot(self.theta_tp,inVec))))>=0.5)[0][0]
        
        if skull:
            self.placeSkull()
        else:
            self.placeFlower()
        
        return skull
    
    def placeTile(self,externalInfo):
        if any(self.placedTiles):
            self.placeFlower()
        else:
            self.chooseTileToPlace(externalInfo)
    
    def betOrPlace(self,externalInfo):
        """
        Function to decide to start the betting or place another tile
        Same input variables as choosing what to place and 
        -include if they have placed a skull already (binary) [1] have placed a skull
        EXTRA
        - random float for "randomness"
        - bais term
        
        RETURNS
        boolean. true, make a bet, false, place a tile
        """
        inVec=np.reshape(y.externalInfo,(-1,1))
        inVec=np.concatenate((inVec,np.array([[int(any(self.placedTiles))],[rand()],[1]])))
        
        betOrPlace = ((1/(1+np.exp(-np.dot(self.theta_bp,inVec))))>=0.5)[0][0]
        
        if betOrPlace:
            #whatToBet
            print("make a bet")
            pass
        else:
            self.placeTile(externalInfo)
            
    def whatToBet(self,externalInfo,betRec):
        """
        Uses the board state record
        """
        inVec=np.concatenate((np.reshape(y.externalInfo,(-1,1)),np.reshape(betRec,(-1,1)),np.array([[int(any(self.placedTiles))],[rand()],[1]])))
        availableBets=np.zeros((1,12))
        availableBets[:,0:sum(externalInfo[:,2])]=1
        availableBets[:,0:max(np.where(betRec.any(axis=0))[0])+1]=0
        inVec=np.concatenate((inVec,availableBets.T))
        bet=np.argmax((1/(1+np.exp(-np.dot(self.theta_w2b,inVec))))*availableBets.T)+1
        return bet

Game glass

In [111]:
class skullGame:
    def __init__(self,player1,player2,player3,player4):
        self.playerArray = [player1,player2,player3,player4]
        self.firstPlayerOfRound = random.randint(0,len(self.playerArray)-1)
        self.externalInfo = np.array([[0,4,0,1,0,((i-self.firstPlayerOfRound)%4)+1,0] for i in range(1,5)])
        self.initPlacement()
        self.betRec=np.zeros((len(self.playerArray),(len(self.playerArray)*3)+1))
        
    def initPlacement(self):
        for ind in range(0,len(self.playerArray)):
            self.playerArray[ind].chooseTileToPlace(self.externalInfo[list(range(0,ind))+list(range(ind+1,len(y.playerArray)))+[ind],:])
            self.externalInfo[ind][2]=1

## Testing zone

In [112]:
x=skullPlayer()

In [113]:
x.summary()

Flowers: 3
Skull: 1
Point: 0
Placed tiles: []


In [114]:
x.placeFlower()
x.placedTiles

[False]

In [115]:
x.placeSkull()
x.placedTiles

[False, True]

In [116]:
x.placeSkull()
x.placedTiles

[False, True]

In [117]:
x.removeRandom()
x.summary()

Flowers: 2
Skull: 1
Point: 0
Placed tiles: []


In [118]:
x.placedTiles

[]

In [119]:
playerList=[skullPlayer() for x in range(0,4)]

In [120]:
y=skullGame(*playerList)

In [121]:
y.externalInfo

array([[0, 4, 1, 1, 0, 4, 0],
       [0, 4, 1, 1, 0, 1, 0],
       [0, 4, 1, 1, 0, 2, 0],
       [0, 4, 1, 1, 0, 3, 0]])

In [122]:
y.betRec.shape

(4, 13)

In [107]:
for player in playerList:
    player.summary()

Flowers: 3
Skull: 1
Point: 0
Placed tiles: [False]
Flowers: 3
Skull: 1
Point: 0
Placed tiles: [False]
Flowers: 3
Skull: 1
Point: 0
Placed tiles: [True]
Flowers: 3
Skull: 1
Point: 0
Placed tiles: [True]


In [38]:
any(playerList[-1].placedTiles)

False

In [66]:
ind=3
y.externalInfo[list(range(0,ind))+list(range(ind+1,len(y.playerArray)))+[ind],:]

array([[0, 4, 1, 1, 0, 4, 0],
       [0, 4, 1, 1, 0, 1, 0],
       [0, 4, 1, 1, 0, 2, 0],
       [0, 4, 1, 1, 0, 3, 0]])

In [63]:
y.externalInfo

array([[0, 4, 1, 1, 0, 4, 0],
       [0, 4, 1, 1, 0, 1, 0],
       [0, 4, 1, 1, 0, 2, 0],
       [0, 4, 1, 1, 0, 3, 0]])

In [73]:
int(any(player.placedTiles))

0

In [70]:
type(1)

int

In [79]:
pd.DataFrame(y.externalInfo, 
             index=["player1","player2","player3","player4"],
             columns=["has_point","tiles","tiles_placed","skull_has","skull_none","turn_order","out"])

,has_point,tiles,tiles_place,skull_has,skull_none,turn_order,out
player1,0,4,1,1,0,4,0
player2,0,4,1,1,0,1,0
player3,0,4,1,1,0,2,0
player4,0,4,1,1,0,3,0


In [80]:
#per player
4*3 #binary entry for each possible bet, entry of 1 if the player has made it.
1 #binary entry if the player has passed
#extra
4*3 # binary available bets that could be made at this point
1 #bias term 
1 #randomness term

12

In [150]:
y.externalInfo.shape

(4, 7)

In [151]:
y.betRec.shape

(4, 13)

In [154]:
4*(7+13)+3+12

95

In [165]:
np.argmax((1/(1+np.exp(-np.ones(3))))*np.array([0,1,0]))

1